In [1]:
from collections import defaultdict
import math
import numpy as np
from numpy import ma
import datetime, time
from datetime import datetime
import netCDF4
from netCDF4 import Dataset, num2date
from scipy.interpolate import griddata

import TomTom
import TomTom.Mesh_maker as Mesh_maker
import TomTom.Functions as Functions
import TomTom.Calc_path as Calc_path

import matplotlib.pyplot as plt
plt.style.use('ggplot')
% matplotlib inline

import os
from cartopy import config
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# User defined values

In [2]:
x_domain = (4.5,5.5)
y_domain =(52.75,53.75)

nl = (1,1)
dx_min = 0.05
blend = 0.8


number_of_neighbor_layers = 1
vship = np.array([3,4,4.5,5,5.5,6 ])
WD_min = 2
Load_flow = Mesh_maker.flow_2D_FM_100m_tot

name_textfile_flow = 'D:/DCSM-FM_100m/A06_pieter/maaktnietuit'
name_textfile_save = 'D:/DCSM-FM_100m/A06_pieter/Rm_Neightbourlayers={},WD_min={},nl={}'.format(number_of_neighbor_layers,WD_min,(nl[0],nl[1],blend))

In [3]:
start_time = time.time()  #12:34

graph = Mesh_maker.Graph_flow_model(name_textfile_flow, 
                                    dx_min, 
                                    blend, 
                                    nl, 
                                    number_of_neighbor_layers, 
                                    vship, 
                                    Load_flow, 
                                    WD_min
                                   )
stop_time = time.time()
computation_time = stop_time - start_time
print("the computational time is:", round(computation_time,2), "sec")

1/7
2/7
3/7
4/7
5/7
6/7
7/7


MemoryError: 

In [ ]:
TomTom.save_object(graph, name_textfile_save)

# Plots

In [ ]:
x_r = np.arange(3,7, 0.001)
y_r = np.arange(52,55, 0.01)
y_r, x_r = np.meshgrid(y_r,x_r)

LS_r = griddata((graph.flow.nodes[:,1], graph.flow.nodes[:,0]), graph.LS, (x_r, y_r), method= 'linear')
WD_r = griddata((graph.flow.nodes[:,1], graph.flow.nodes[:,0]), graph.flow.WD[37], (x_r, y_r), method= 'linear')

In [ ]:
fig = plt.figure(figsize=(15, 15))
ax = plt.axes(projection=ccrs.Mercator())

ax.coastlines(resolution='10m', color='black', linewidth=1)
ax.gridlines(color = 'grey', zorder = 3)
ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', edgecolor='face', facecolor='palegoldenrod'))

cval = np.arange(0,1, 0.2)
plt.contourf(x_r,y_r, LS_r, cval, transform=ccrs.PlateCarree())
cbar = plt.colorbar()

plt.triplot(graph.nodes[:,1], graph.nodes[:,0], graph.tria.simplices, linewidth = 0.5, color = 'k', label = 'Delauney edges', transform=ccrs.PlateCarree())

In [ ]:
curl = np.arange(0,100,0.001)
def LS(nonlinearity):
    LS = 1/(1+curl)**nonlinearity
    return LS

fig = plt.figure()
fig.set_size_inches(6, 6)
plt.semilogx(curl, LS(0), label = r'$\beta =  0$')
plt.semilogx(curl, LS(1), label = r'$\beta =  1$')
plt.semilogx(curl, LS(5), label = r'$\beta =  5$')
plt.semilogx(curl, LS(10), label = r'$\beta =  10$')
plt.semilogx(curl, LS(20), label = r'$\beta =  20$')
plt.semilogx(curl, LS(np.inf), label = r'$\beta =  infinite$')

plt.title("influence of the nonlinearity on the length scale, \n LS = 1/(1+curl)^nonlinearity")
plt.xlabel("Absolute value of the curl or magnitude of the flow (on log scale)")
plt.ylabel("Length scale for different nonlinearities")

plt.legend(loc = 'best')
#plt.savefig("Influence_of_nonlinearity", dpi = 200)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 15))
ax = plt.axes(projection=ccrs.Mercator())

ax.coastlines(resolution='10m', color='black', linewidth=1)
ax.gridlines(color = 'grey', zorder = 3)
ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', edgecolor='face', facecolor='palegoldenrod'))

cval = np.arange(2,50, 0.5)
plt.contourf(x_r,y_r, WD_r, cval, transform=ccrs.PlateCarree())
cbar = plt.colorbar()

plt.plot(graph.nodes[:,1],graph.nodes[:,0],'r.', transform=ccrs.PlateCarree())

In [ ]:
len(graph.nodes) ** 0.5, len(graph.flow.nodes[:,1])**0.5

In [ ]:
fig = plt.figure(figsize=(15, 15))
ax = plt.axes(projection=ccrs.Mercator())

ax.coastlines(resolution='10m', color='black', linewidth=1)
ax.gridlines(color = 'grey', zorder = 3)
ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', edgecolor='face', facecolor='palegoldenrod'))

cval = np.arange(0,50, 0.5)
plt.contourf(x_r,y_r, WD_r, cval, transform=ccrs.PlateCarree())
cbar = plt.colorbar()

#plt.plot(graph.flow.nodes[:,1],graph.flow.nodes[:,0],'r.', transform=ccrs.PlateCarree())

In [ ]:
graph.nodes_index